# Estimate foreground mask in 2D image

# Setup environment

In [40]:
%matplotlib inline

import numpy as np
import glob

from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [42]:
from importlib import reload

import utils
reload(utils)
from utils import *

import unet
reload(unet)
from unet import generate_model

# Parameter setting

In [43]:
matrix_size = (320, 400)
num_classes = 2
num_channel = 3

model_filename = 'models/weights.h5'
monitor = 'val_loss'
nb_epoch = 50
validation_split = 0.1

# Prepare data

In [45]:
imgs_resized = []
masks_resized = []

count_case = 0
for fn_img in glob.glob('datasets/img*.jpg'):
    img_ID = get_img_ID(fn_img)
    fn_mask = 'datasets/mask' + img_ID + '.jpg'
    
    # Load image
    img = load_data(fn_img)
    
    # Resize image
    img_resized = resize(img, matrix_size)
    
    # Load mask
    mask = load_data(fn_mask, flag_mask=True)
    
    # Resize mask
    mask_resized = resize(mask[np.newaxis,:], matrix_size)[0,:,:]

    imgs_resized.append(img_resized)
    masks_resized.append(mask_resized)
    
    count_case += 1
    if count_case%100 == 0:
        print('Loaded {} cases'.format(count_case))
    
imgs_resized = np.stack(imgs_resized, axis=0)
masks_resized = np.stack(masks_resized, axis=0)

Loaded 100 cases
Loaded 200 cases


In [46]:
# Convert mask to one-hot label
labels_resized = np_utils.to_categorical(masks_resized, num_classes)
del masks_resized

In [48]:
# Split into training/test dataset
data_train, data_test, label_train, label_test = train_test_split(imgs_resized, labels_resized, test_size = 0.1)

# Training stage

## Prepare model (U-Net) 

In [10]:
# Build U-Net model
model = generate_model(num_classes, num_channel, input_size=matrix_size, output_size=matrix_size)

## Configure callback

In [12]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

# Model checkpoint to save the training results
checkpointer = ModelCheckpoint(
    filepath=model_filename,
    monitor=monitor,
    verbose=0,
    save_best_only=True,
    save_weights_only=True)

callbacks = [checkpointer]

## Start training 

In [ ]:
K.set_value(model.optimizer.lr, 1e-4)

history = model.fit(
    data_train,
    label_train,
    batch_size=5,
    epochs=nb_epoch,
    validation_split=validation_split,
    verbose=1,
    callbacks=callbacks)

## Summary of training

In [ ]:
# summarize history for accuracy
plt.figure()
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Test stage

## Load optimal model

In [51]:
model = generate_model(num_classes, num_channel, input_size=matrix_size, output_size=matrix_size)

model.load_weights(model_filename)
#model.load_weights('models/weights_optimal.h5')

## Inference on test data

In [ ]:
infers_test = []

for img_test in data_test:
    
    pred = model.predict(img_test[np.newaxis,:], verbose=1)
    pred_classes = np.argmax(pred, axis=-1)
    
    mask_test = pred_classes

    infers_test.append(mask_test)

infers_test = np.stack(infers_test, axis=0)

## Calculate metric (Dice)

In [ ]:
scores_dice = []

for i in range(len(data_test)):
    mask_pred = infers_test[i,0,:,:]
    mask_true = label_test[i,:,:,1]
    
    dice = calc_dice(mask_pred, mask_true)
    #print(i, dice)
    
    scores_dice.append(dice)
    
print('Averaged DICE score on {0} test case: {1:.3f}'.format(len(scores_dice), np.asarray(scores_dice).mean()))